In [2]:
import numpy as np
import pandas as pd


In [6]:
data = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


We see that there are unwanted columns in our data so we'll just remove that.

In [7]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [8]:
train=pd.DataFrame()
train['sms']=data['v2']
train['label']=data['v1']
train

,sms,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


let's see some data analysis 

In [9]:
train['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

First we will continue with this imbalanced data only 

In [10]:
import random
for i in range(5):
    index = random.randint(0,len(train['sms']))
    print(str(index)+'th sms is: '+train['sms'][index])

4234th sms is: FREEMSG: Our records indicate you may be entitled to 3750 pounds for the Accident you had. To claim for free reply with YES to this msg. To opt out text STOP
2520th sms is: Misplaced your number and was sending texts to your old number. Wondering why i've not heard from you this year. All the best in your mcat. Got this number from my atlanta friends
860th sms is: In work now. Going have in few min.
3866th sms is: Alright we're hooked up, where you guys at
3364th sms is: Sorry, I'll call later


In [17]:
pip! install nltk

ERROR: unknown command "!"
Note: you may need to restart the kernel to use updated packages.


In [21]:
#now we will clean n data a bit
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
wordnet=WordNetLemmatizer()

In [26]:
for i,sentence in enumerate(train['sms']):
    new_sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    train['sms'][i]=new_sentence

Now we removed characters other than alphabets

In [28]:
for i in range(5):
    index = random.randint(0,len(train['sms']))
    print(str(index)+'th sms is: '+train['sms'][index])

1347th sms is: She doesnt need any test 
988th sms is: Yun ah the ubi one say if    wan call by tomorrow call          look for irene ere only got bus                   Ubi cres ubi tech park  ph for  st  wkg days   n
3581th sms is: You are right  Meanwhile how s project twins comin up
4092th sms is:  Life is nothing wen v get everything    But   life is everything wen v miss something     Real value of people wil be realized only in their absence     gud mrng 
3810th sms is: Excellent  Wish we were together right now 


In [30]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [31]:
corpus = []
for i in range(len(train['sms'])):
    review = train['sms'][i].lower()
    review = review.split()
    review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [33]:
len(corpus)==len(train['sms'])

True

In [34]:
#Let's checkout some lemmatized texts
for i in range(5):
    index = random.randint(0,len(train['sms']))
    print(str(index)+'th sms is: '+corpus[index])

320th sms is: merry christmas babe love ya kiss
1491th sms is: account credited free text message activate txt word credit c www biz
3565th sms is: ok come home vl nice meet v chat
90th sms is: yeah stand close tho catch something
4243th sms is: aight chillin friend room text way


In [35]:
#let's vectorize them
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
transformer =TfidfVectorizer(max_features=500)
X = transformer.fit_transform(corpus).toarray()

In [42]:
X.shape

(5572, 500)

In [46]:
np.unique(X[0])

array([0.        , 0.31623729, 0.33593142, 0.39583446, 0.40037645,
       0.48036757, 0.48929013])

In [47]:
Y = train['label'].apply(lambda x: 1 if x=='ham' else 0)
Y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: label, Length: 5572, dtype: int64

In [55]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X,Y)
model.score(X,Y)

0.9545944005743001

In [52]:
y_predicted = model.predict(X)
y_predicted

array([1, 1, 0, ..., 1, 1, 1])

In [53]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(Y, y_predicted)
cm 

array([[ 501,  246],
       [   7, 4818]])

In [57]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X,Y)
model_rf.score(X,Y)

0.999102656137832

In [58]:
y_predicted = model_rf.predict(X)
y_predicted

array([1, 1, 0, ..., 1, 1, 1])

In [59]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(Y, y_predicted)
cm 

array([[ 742,    5],
       [   0, 4825]])

Now that's some impressive prediction we got

In [60]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

In [62]:
filename = '/kaggle/working/RFClassifier_model.sav'
pickle.dump(model_rf, open(filename, 'wb'))

In [63]:
filename = '/kaggle/working/KNNClassifier_model.sav'
pickle.dump(model, open(filename, 'wb'))